In [7]:
import pandas as pd

## ARX

In [8]:
from lwautils import lwa_arx
ma = lwa_arx.ARX()
adrs = [17,21,27,31]  # currently installed

/home/ubuntu/anaconda3/envs/casa/lib/python3.6/site-packages/lwautils/conf/etcdConfig.yml


In [9]:
arxcfg = []
for adr in adrs:
    dd = ma.get_chan_cfg(adr, 0)
    dd['adr'] = adr
    arxcfg.append(dd)
        
df_arx = pd.DataFrame.from_dict(arxcfg)
df_arx.set_index('adr', inplace=True)

In [10]:
df_arx

,sig_on,narrow_lpf,narrow_hpf,first_atten,second_atten,dc_on
adr,,,,,,
17,True,False,False,7.5,15.0,True
21,True,False,False,7.5,15.0,True
27,True,False,False,7.5,15.0,True
31,True,False,False,7.5,15.0,True


## F-engine

In [11]:
from lwa_f import snap2_fengine, snap2_feng_etcd_client

In [12]:
snap2s = ['snap01']
lwa_f = snap2_fengine.Snap2Fengine('snap01')
lwa_fe = snap2_feng_etcd_client.Snap2FengineEtcdClient('snap01', 1)

2021-08-25 22:01:20,369 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2021-08-25 22:01:20,677 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2021-08-25 22:01:20,981 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
2021-08-25 22:01:21,380 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2021-08-25 22:01:21,680 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2021-08-25 22:01:21,980 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
